In [16]:
import pandas as pd
pd.set_option("display.max_rows", None)

In [18]:
import os

directory = f'data/ebm_nlp_2_00/documents/'
for filename in os.scandir(directory):
    if filename.path.endswith(".txt"):
        print(filename.path)
        print(filename.name)
        print(os.path.splitext(filename.path)[0]+'.tokens')
        print(os.path.splitext(filename.name)[0]+'.tokens')

data/ebm_nlp_2_00/documents/2187009.txt
2187009.txt
data/ebm_nlp_2_00/documents/2187009.tokens
2187009.tokens
data/ebm_nlp_2_00/documents/18485420.txt
18485420.txt
data/ebm_nlp_2_00/documents/18485420.tokens
18485420.tokens
data/ebm_nlp_2_00/documents/3548346.txt
3548346.txt
data/ebm_nlp_2_00/documents/3548346.tokens
3548346.tokens
data/ebm_nlp_2_00/documents/21724047.txt
21724047.txt
data/ebm_nlp_2_00/documents/21724047.tokens
21724047.tokens
data/ebm_nlp_2_00/documents/8213019.txt
8213019.txt
data/ebm_nlp_2_00/documents/8213019.tokens
8213019.tokens
data/ebm_nlp_2_00/documents/25488965.txt
25488965.txt
data/ebm_nlp_2_00/documents/25488965.tokens
25488965.tokens
data/ebm_nlp_2_00/documents/22395144.txt
22395144.txt
data/ebm_nlp_2_00/documents/22395144.tokens
22395144.tokens
data/ebm_nlp_2_00/documents/23365106.txt
23365106.txt
data/ebm_nlp_2_00/documents/23365106.tokens
23365106.tokens
data/ebm_nlp_2_00/documents/17174705.txt
17174705.txt
data/ebm_nlp_2_00/documents/17174705.tokens
17

data/ebm_nlp_2_00/documents/22853706.txt
22853706.txt
data/ebm_nlp_2_00/documents/22853706.tokens
22853706.tokens
data/ebm_nlp_2_00/documents/26510728.txt
26510728.txt
data/ebm_nlp_2_00/documents/26510728.tokens
26510728.tokens
data/ebm_nlp_2_00/documents/18541792.txt
18541792.txt
data/ebm_nlp_2_00/documents/18541792.tokens
18541792.tokens
data/ebm_nlp_2_00/documents/8570776.txt
8570776.txt
data/ebm_nlp_2_00/documents/8570776.tokens
8570776.tokens
data/ebm_nlp_2_00/documents/8964729.txt
8964729.txt
data/ebm_nlp_2_00/documents/8964729.tokens
8964729.tokens
data/ebm_nlp_2_00/documents/2139523.txt
2139523.txt
data/ebm_nlp_2_00/documents/2139523.tokens
2139523.tokens
data/ebm_nlp_2_00/documents/21690083.txt
21690083.txt
data/ebm_nlp_2_00/documents/21690083.tokens
21690083.tokens
data/ebm_nlp_2_00/documents/17387189.txt
17387189.txt
data/ebm_nlp_2_00/documents/17387189.tokens
17387189.tokens
data/ebm_nlp_2_00/documents/8707361.txt
8707361.txt
data/ebm_nlp_2_00/documents/8707361.tokens
87073

data/ebm_nlp_2_00/documents/2975995.txt
2975995.txt
data/ebm_nlp_2_00/documents/2975995.tokens
2975995.tokens
data/ebm_nlp_2_00/documents/9855086.txt
9855086.txt
data/ebm_nlp_2_00/documents/9855086.tokens
9855086.tokens
data/ebm_nlp_2_00/documents/17051443.txt
17051443.txt
data/ebm_nlp_2_00/documents/17051443.tokens
17051443.tokens
data/ebm_nlp_2_00/documents/8801151.txt
8801151.txt
data/ebm_nlp_2_00/documents/8801151.tokens
8801151.tokens
data/ebm_nlp_2_00/documents/12676060.txt
12676060.txt
data/ebm_nlp_2_00/documents/12676060.tokens
12676060.tokens
data/ebm_nlp_2_00/documents/19371522.txt
19371522.txt
data/ebm_nlp_2_00/documents/19371522.tokens
19371522.tokens
data/ebm_nlp_2_00/documents/9864127.txt
9864127.txt
data/ebm_nlp_2_00/documents/9864127.tokens
9864127.tokens
data/ebm_nlp_2_00/documents/19591981.txt
19591981.txt
data/ebm_nlp_2_00/documents/19591981.tokens
19591981.tokens
data/ebm_nlp_2_00/documents/11701672.txt
11701672.txt
data/ebm_nlp_2_00/documents/11701672.tokens
117016

data/ebm_nlp_2_00/documents/22502844.txt
22502844.txt
data/ebm_nlp_2_00/documents/22502844.tokens
22502844.tokens
data/ebm_nlp_2_00/documents/1376306.txt
1376306.txt
data/ebm_nlp_2_00/documents/1376306.tokens
1376306.tokens
data/ebm_nlp_2_00/documents/11495215.txt
11495215.txt
data/ebm_nlp_2_00/documents/11495215.tokens
11495215.tokens
data/ebm_nlp_2_00/documents/24898665.txt
24898665.txt
data/ebm_nlp_2_00/documents/24898665.tokens
24898665.tokens
data/ebm_nlp_2_00/documents/18397386.txt
18397386.txt
data/ebm_nlp_2_00/documents/18397386.tokens
18397386.tokens
data/ebm_nlp_2_00/documents/23095369.txt
23095369.txt
data/ebm_nlp_2_00/documents/23095369.tokens
23095369.tokens
data/ebm_nlp_2_00/documents/11863211.txt
11863211.txt
data/ebm_nlp_2_00/documents/11863211.tokens
11863211.tokens
data/ebm_nlp_2_00/documents/8463520.txt
8463520.txt
data/ebm_nlp_2_00/documents/8463520.tokens
8463520.tokens
data/ebm_nlp_2_00/documents/6686808.txt
6686808.txt
data/ebm_nlp_2_00/documents/6686808.tokens
6

data/ebm_nlp_2_00/documents/14668075.txt
14668075.txt
data/ebm_nlp_2_00/documents/14668075.tokens
14668075.tokens
data/ebm_nlp_2_00/documents/10719133.txt
10719133.txt
data/ebm_nlp_2_00/documents/10719133.tokens
10719133.tokens
data/ebm_nlp_2_00/documents/12182253.txt
12182253.txt
data/ebm_nlp_2_00/documents/12182253.tokens
12182253.tokens
data/ebm_nlp_2_00/documents/20509069.txt
20509069.txt
data/ebm_nlp_2_00/documents/20509069.tokens
20509069.tokens
data/ebm_nlp_2_00/documents/15562654.txt
15562654.txt
data/ebm_nlp_2_00/documents/15562654.tokens
15562654.tokens
data/ebm_nlp_2_00/documents/7559252.txt
7559252.txt
data/ebm_nlp_2_00/documents/7559252.tokens
7559252.tokens
data/ebm_nlp_2_00/documents/22881991.txt
22881991.txt
data/ebm_nlp_2_00/documents/22881991.tokens
22881991.tokens
data/ebm_nlp_2_00/documents/20204689.txt
20204689.txt
data/ebm_nlp_2_00/documents/20204689.tokens
20204689.tokens
data/ebm_nlp_2_00/documents/25617202.txt
25617202.txt
data/ebm_nlp_2_00/documents/25617202.t

...

19912211.txt
data/ebm_nlp_2_00/documents/19912211.tokens
19912211.tokens
data/ebm_nlp_2_00/documents/4220015.txt
4220015.txt
data/ebm_nlp_2_00/documents/4220015.tokens
4220015.tokens
data/ebm_nlp_2_00/documents/9703286.txt
9703286.txt
data/ebm_nlp_2_00/documents/9703286.tokens
9703286.tokens
data/ebm_nlp_2_00/documents/11205419.txt
11205419.txt
data/ebm_nlp_2_00/documents/11205419.tokens
11205419.tokens
data/ebm_nlp_2_00/documents/24661050.txt
24661050.txt
data/ebm_nlp_2_00/documents/24661050.tokens
24661050.tokens
data/ebm_nlp_2_00/documents/1319876.txt
1319876.txt
data/ebm_nlp_2_00/documents/1319876.tokens
1319876.tokens
data/ebm_nlp_2_00/documents/15719741.txt
15719741.txt
data/ebm_nlp_2_00/documents/15719741.tokens
15719741.tokens
data/ebm_nlp_2_00/documents/6653648.txt
6653648.txt
data/ebm_nlp_2_00/documents/6653648.tokens
6653648.tokens
data/ebm_nlp_2_00/documents/9322632.txt
9322632.txt
data/ebm_nlp_2_00/documents/9322632.tokens
9322632.tokens
data/ebm_nlp_2_00/documents/9570244

data/ebm_nlp_2_00/documents/8015504.txt
8015504.txt
data/ebm_nlp_2_00/documents/8015504.tokens
8015504.tokens
data/ebm_nlp_2_00/documents/15929820.txt
15929820.txt
data/ebm_nlp_2_00/documents/15929820.tokens
15929820.tokens
data/ebm_nlp_2_00/documents/9917050.txt
9917050.txt
data/ebm_nlp_2_00/documents/9917050.tokens
9917050.tokens
data/ebm_nlp_2_00/documents/7844976.txt
7844976.txt
data/ebm_nlp_2_00/documents/7844976.tokens
7844976.tokens
data/ebm_nlp_2_00/documents/3516608.txt
3516608.txt
data/ebm_nlp_2_00/documents/3516608.tokens
3516608.tokens
data/ebm_nlp_2_00/documents/10636373.txt
10636373.txt
data/ebm_nlp_2_00/documents/10636373.tokens
10636373.tokens
data/ebm_nlp_2_00/documents/9278836.txt
9278836.txt
data/ebm_nlp_2_00/documents/9278836.tokens
9278836.tokens
data/ebm_nlp_2_00/documents/16920077.txt
16920077.txt
data/ebm_nlp_2_00/documents/16920077.tokens
16920077.tokens
data/ebm_nlp_2_00/documents/9895371.txt
9895371.txt
data/ebm_nlp_2_00/documents/9895371.tokens
9895371.token

data/ebm_nlp_2_00/documents/6350412.txt
6350412.txt
data/ebm_nlp_2_00/documents/6350412.tokens
6350412.tokens
data/ebm_nlp_2_00/documents/18711222.txt
18711222.txt
data/ebm_nlp_2_00/documents/18711222.tokens
18711222.tokens
data/ebm_nlp_2_00/documents/18852963.txt
18852963.txt
data/ebm_nlp_2_00/documents/18852963.tokens
18852963.tokens
data/ebm_nlp_2_00/documents/15486373.txt
15486373.txt
data/ebm_nlp_2_00/documents/15486373.tokens
15486373.tokens
data/ebm_nlp_2_00/documents/7946102.txt
7946102.txt
data/ebm_nlp_2_00/documents/7946102.tokens
7946102.tokens
data/ebm_nlp_2_00/documents/17286730.txt
17286730.txt
data/ebm_nlp_2_00/documents/17286730.tokens
17286730.tokens
data/ebm_nlp_2_00/documents/21887489.txt
21887489.txt
data/ebm_nlp_2_00/documents/21887489.tokens
21887489.tokens
data/ebm_nlp_2_00/documents/10073522.txt
10073522.txt
data/ebm_nlp_2_00/documents/10073522.tokens
10073522.tokens
data/ebm_nlp_2_00/documents/15610252.txt
15610252.txt
data/ebm_nlp_2_00/documents/15610252.token

20356395.tokens
data/ebm_nlp_2_00/documents/16030622.txt
16030622.txt
data/ebm_nlp_2_00/documents/16030622.tokens
16030622.tokens
data/ebm_nlp_2_00/documents/16413892.txt
16413892.txt
data/ebm_nlp_2_00/documents/16413892.tokens
16413892.tokens
data/ebm_nlp_2_00/documents/21800228.txt
21800228.txt
data/ebm_nlp_2_00/documents/21800228.tokens
21800228.tokens
data/ebm_nlp_2_00/documents/22277402.txt
22277402.txt
data/ebm_nlp_2_00/documents/22277402.tokens
22277402.tokens
data/ebm_nlp_2_00/documents/10665129.txt
10665129.txt
data/ebm_nlp_2_00/documents/10665129.tokens
10665129.tokens
data/ebm_nlp_2_00/documents/23768813.txt
23768813.txt
data/ebm_nlp_2_00/documents/23768813.tokens
23768813.tokens
data/ebm_nlp_2_00/documents/17414940.txt
17414940.txt
data/ebm_nlp_2_00/documents/17414940.tokens
17414940.tokens
data/ebm_nlp_2_00/documents/6307704.txt
6307704.txt
data/ebm_nlp_2_00/documents/6307704.tokens
6307704.tokens
data/ebm_nlp_2_00/documents/10770301.txt
10770301.txt
data/ebm_nlp_2_00/docu

data/ebm_nlp_2_00/documents/3923528.txt
3923528.txt
data/ebm_nlp_2_00/documents/3923528.tokens
3923528.tokens
data/ebm_nlp_2_00/documents/18073312.txt
18073312.txt
data/ebm_nlp_2_00/documents/18073312.tokens
18073312.tokens
data/ebm_nlp_2_00/documents/6189335.txt
6189335.txt
data/ebm_nlp_2_00/documents/6189335.tokens
6189335.tokens
data/ebm_nlp_2_00/documents/1712835.txt
1712835.txt
data/ebm_nlp_2_00/documents/1712835.tokens
1712835.tokens
data/ebm_nlp_2_00/documents/26139208.txt
26139208.txt
data/ebm_nlp_2_00/documents/26139208.tokens
26139208.tokens
data/ebm_nlp_2_00/documents/25912590.txt
25912590.txt
data/ebm_nlp_2_00/documents/25912590.tokens
25912590.tokens
data/ebm_nlp_2_00/documents/25714521.txt
25714521.txt
data/ebm_nlp_2_00/documents/25714521.tokens
25714521.tokens
data/ebm_nlp_2_00/documents/1350581.txt
1350581.txt
data/ebm_nlp_2_00/documents/1350581.tokens
1350581.tokens
data/ebm_nlp_2_00/documents/24887304.txt
24887304.txt
data/ebm_nlp_2_00/documents/24887304.tokens
248873

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Read in the dataframe containing entire paper abstracts (NOT pre-split into sentences)
df = pd.read_csv(f'data/nlp-align_{search_word}.csv')

# Create the outputs directory for this search word
Path(f"outputs/{search_word}").mkdir(parents=True, exist_ok=True)

# Split apart the 'Title' and 'Abstract' columns, add period to 'Title' if not present
def separate_title_abstract(group):
    row = group.loc[0]
    abs_text = tokenize.sent_tokenize(row['Abstract'])
    return pd.DataFrame({
        'URL': [row['URL']] * 2,
        'ID': [row['ID']] * 2,
        'Type': ['Title', 'Abstract'],
        'Text': [row['Title']+'.' if not row['Title'].endswith('.') else row['Title'], 
                 row['Abstract']]
    })

# Restructure the dataframe to be more usable...
df = df.groupby('ID', group_keys=False).apply(
    lambda row: separate_title_abstract(row)
).reset_index(drop=True)

df